In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from branca.element import Figure
fig=Figure(width=550,height=350)
plt.rcParams['figure.figsize']=10,10

In [2]:
df1=pd.read_csv("nhai_take_home_final.csv")
toll_prices=pd.read_excel('Road Condition and Other Details.xlsx',sheet_name=1)
road_data1=pd.read_excel('Road Condition and Other Details.xlsx',sheet_name=2,skiprows=1)
road_data2=pd.read_excel('Road Condition and Other Details.xlsx',sheet_name=3,skiprows=1)

#importing the data from sheets and csv using pandas


In [3]:
print(df1.columns)
print(toll_prices.columns)
print(road_data1.columns)
print(road_data2.columns)

Index(['Unnamed: 0', 'Toll', 'Vehicle', 'Type', 'trxtime'], dtype='object')
Index(['Entry', 'Exit', 'Car', 'Tempo', 'Truck'], dtype='object')
Index(['NHNumber', 'Description', 'Chainage', 'Latitude', 'Longitude',
       'Altitude', 'SurveyDate', 'State', 'District', 'EntrustedTo', 'ROName',
       'OldNHNumber'],
      dtype='object')
Index(['NHNumber', 'SectionCode', 'StartChainage', 'EndChainage', 'Direction',
       'LaneNumber', 'LwpIri', 'RwpIri', 'LaneIRI', 'Latitude', 'Longitude',
       'Latitude.1', 'Longitude.1'],
      dtype='object')


In [4]:

df1.groupby('Toll')["Unnamed: 0"].value_counts()
#since we are seeing that the data Unnamed:0 is having some relevance but dont have the relevance in the current analysis we are dropping the column 
df1=df1.drop(columns=["Unnamed: 0"])

In [5]:
#get the count of different type of vehicles across Tolls
df1.groupby('Type')["Toll"].value_counts()

Type   Toll
Car    B       6623
       C       6623
       D       6623
       A       6460
Tempo  B       6695
       C       6695
       D       6695
       A       6514
Truck  B       6680
       C       6680
       D       6680
       A       6509
Name: Toll, dtype: int64

### Q1:Visualize the highway alignment within the Jupyter Notebook along with the location and labels of the Toll Plazas.

**We are using folium to visualize the maps and the alignment of the road**
* The understanding that can be taken for alignment for  this usecase is the data has longitude and latitude of the data but each point of latitude and longitude when plotted on the graph will give me allignment of the road how is the path of the road
* How are the turns and curve in a road can also defined as alignment since we dont have specific allignment formula which can be calculated based on the longitude and latitude of the road.

**Creating pair of Latitude and Longitude for Folium**

In [6]:
road_data1_location=road_data1.apply(lambda row: pd.Series([(row['Latitude'], row['Longitude'])]), axis=1)[0].tolist()

### Creating a base map

In [7]:
fig5=Figure(height=550,width=750)
m5=folium.Map(location=[road_data1["Latitude"].min(), road_data1["Longitude"].min()],zoom_start=14)


In [8]:
f1=folium.FeatureGroup("Road")
line_1=folium.vector_layers.PolyLine(road_data1_location,popup='<b>Path of Vehicle_1</b>',tooltip='Vehicle_1',color='blue',weight=3).add_to(f1)
folium.Marker([25.60, 75.00],popup = 'KadiSahena Toll Plaza').add_to(m5)
folium.Marker([25.8, 75.9],popup = 'Pallai Toll Plaza').add_to(m5)


 
#folium.Marker([28.5011226, 77.4099794],
 #             popup = 'GeeksforGeeks').add_to(my_map4)
f1.add_to(m5)
folium.LayerControl().add_to(m5)
m5

### Q 2: There have been reports that people have been able bypass toll-plazas on entry and it is leading to losses of revenue. There are two toll-able segments in the data (AB/BA) and (CD/DC). The toll is to be a deducted at the entry to a segment. Sometimes the toll-plaza lets some people go without deducting their entry or people have been able to find an unofficial bypass allowing them to skip and entry gate.
* a. Investigate if there is any toll-leakage, if so at which segment/toll-plaza?
* b. Number of trips which were possibly bypassed toll?
* c. What is the possible revenue loss that is happening pertaining to those trips?

In [9]:
Grouped_Type=df1.groupby(["Type"])["Vehicle"].value_counts().to_frame()
Grouped_Type=Grouped_Type.rename(columns={"Vehicle":"count"})
Grouped_Type=Grouped_Type.reset_index()

**We can see that the count of the Type for each toll is 4 but for some of them has 3 hence we are seeing these are the cars which have bypass the toll unofficial**

#### a. Investigate if there is any toll-leakage, if so at which segment/toll-plaza?

In [13]:
Check_leakage=Grouped_Type[(Grouped_Type["count"]==1)|(Grouped_Type['count']==2)|(Grouped_Type['count']==3)]

In [24]:
prices=[]
car=100
tempo=150
truck=200
for i in Check_leakage["Type"]:
    if i=="Car":
        prices.append(100)
    elif i=='Tempo':
        prices.append(150)
    else:
        prices.append(200)

In [25]:
Check_leakage["prices"]=prices

/var/folders/yz/x7v19kp95bzcpnz2fgpndjp40000gn/T/ipykernel_34273/3132859037.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Check_leakage["prices"]=prices


In [16]:
Toll_Leaker=pd.crosstab(df1.Vehicle,df1.Toll)

In [19]:
Toll_Leaker[Toll_Leaker["A"]==0]

Toll,A,B,C,D
Vehicle,,,,
X10046,0,1,1,1
X10054,0,1,1,1
X10098,0,1,1,1
X10182,0,1,1,1
X10218,0,1,1,1
...,...,...,...,...
X9682,0,1,1,1
X9716,0,1,1,1
X9816,0,1,1,1


In [20]:
Toll_Leaker[Toll_Leaker["B"]==0]

Toll,A,B,C,D
Vehicle,,,,


In [21]:
Toll_Leaker[Toll_Leaker["B"]==0]

Toll,A,B,C,D
Vehicle,,,,


In [26]:
Check_leakage["prices"].sum()

77650

* We can see that the there is leakage in Toll A so this the Toll we should concentrate to get to enquiry
* We can see there are 515 number of trips are being bypassed
* The Revenue of 77650 Rupees is being Lossed from the given toll data

### The speed of a particular vehicle on a particular stretch in a particular direction depends on multiple factors such as the traffic, vehicle type, condition of the road, direction etc.
* a. From the data available, identify average/median/quartile speeds of vehicles on AB and CD stretches in each direction.
* b. Does the speed vary by type of vehicle anywhere?

### Distance Calculator

In [41]:

import geopy.distance
coords_1 = (road_data2["Latitude"].min(),road_data2["Longitude"].min())
coords_2 = (road_data2["Latitude.1"].max(), road_data2["Longitude.1"].max())



150.3314160851677 km


* The distance here is 150.33km 

In [ ]:
df1.info()
#df1.groupby(['Vehicle','Type','Toll']).agg()